In [1]:
GROQ_API_KEY = "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"

PROMPT_TEMPLATE = """
Ответь на вопрос, основываясь исключительно на предоставленном контексте:

{context}

---

Вопрос к данному контексту: {question}

---

Приоритетный язык ответа - русский
"""

# Ollama

In [2]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 11868    0 11868    0     0  40619      0 --:--:-- --:--:-- --:--:-- 40643
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [4]:
%%capture
!ollama pull llama3
!ollama pull phi3
!ollama pull openhermes

!ollama pull snowflake-arctic-embed:137m
!ollama pull nomic-embed-text
!ollama pull mxbai-embed-large

In [3]:
!ollama list

NAME                       	ID          	SIZE  	MODIFIED       
mxbai-embed-large:latest   	468836162de7	669 MB	22 minutes ago	
nomic-embed-text:latest    	0a109f422b47	274 MB	22 minutes ago	
snowflake-arctic-embed:137m	12616299a158	274 MB	23 minutes ago	
openhermes:latest          	95477a2659b7	4.1 GB	23 minutes ago	
phi3:latest                	4f2222927938	2.2 GB	24 minutes ago	
llama3:latest              	365c0bd3c000	4.7 GB	24 minutes ago	


# Gradio

### Libraries

In [6]:
%%capture
!pip install gradio

!pip install chromadb
!pip install python-docx python-dotenv python-pptx
!pip install bs4

!pip install unstructured
!pip install langchain_community langchain_core langchain_text_splitters
!pip install langchain_chroma langchain_openai

!pip install groq

### Imports

In [4]:
import os
import shutil

import gradio as gr

# from langchain_community.vectorstores import Chroma, FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_community.llms.ollama import Ollama

from langchain_community.document_loaders import BSHTMLLoader, CSVLoader, JSONLoader, TextLoader
from langchain_community.document_loaders.word_document import UnstructuredWordDocumentLoader
from langchain_community.document_loaders.excel import UnstructuredExcelLoader
from langchain_community.document_loaders.powerpoint import UnstructuredPowerPointLoader
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_core.documents import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter

from groq import Groq

## Main code

### Prepare Data

In [5]:
def load_documents(files: list[str]):
    documents = []
    for file in files:
        extension = file.split('.')[-1]
        if extension == 'txt':
            documents += TextLoader(file).load()
        if extension == 'csv':
            documents += CSVLoader(file).load()
        if extension == 'html':
            documents += BSHTMLLoader(file, open_encoding='utf-8').load()
        if extension == 'json':
            continue
            documents += JSONLoader(file, jq_schema='.quiz').load()
        if extension == 'pdf':
            documents += PyPDFLoader(file).load()
        if extension == 'xlsx':
            documents += UnstructuredExcelLoader(file).load()
        if extension == 'docx':
            documents += UnstructuredWordDocumentLoader(file).load()
        if extension == 'pptx':
            documents += UnstructuredPowerPointLoader(file).load()
    return documents


In [6]:
def split_documents(documents: list[Document], splitter):
    splitter_chunk_size = int(splitter.split(',')[0].split('=')[1].strip())
    splitter_chunk_overlap = int(splitter.split(',')[1].split('=')[1].strip())
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=splitter_chunk_size,
        chunk_overlap=splitter_chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    print("[INFO] split_documents")
    print("[DEBUG]", documents)
    return text_splitter.split_documents(documents)

In [7]:
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}"
        if page:
            current_page_id += f":{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id
    print("[INFO] calculate_chunk_ids")
    return chunks

### Interface

In [8]:
def process_input(model_name, embeddings, splitter, files, question):
    documents = load_documents(files)

    chunks = split_documents(documents, splitter)
    chunks = calculate_chunk_ids(chunks)
    db = Chroma.from_documents(chunks, OllamaEmbeddings(model=embeddings))

    results = db.similarity_search_with_score(question, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt = PROMPT_TEMPLATE.format(context=context_text, question=question)
    if model_name == "llama3.1":
        client = Groq(
            api_key=GROQ_API_KEY,
        )

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="llama3-70b-8192",
        )

        return chat_completion.choices[0].message.content
    else:
        model = Ollama(model=model_name)
        response_text = model.invoke(prompt)

        return response_text

In [9]:
iface = gr.Interface(fn=process_input,
                     inputs=[
                         gr.Dropdown(label="Model",
                                     choices=[
                                         "llama3",
                                         "phi3",
                                         "openhermes",
                                         "llama3.1"
                                         ],
                                     value="llama3"),
                         gr.Dropdown(label="Embedding",
                                     choices=[
                                         "nomic-embed-text",
                                         "snowflake-arctic-embed:137m",
                                         "mxbai-embed-large"
                                         ],
                                     value="nomic-embed-text"),
                         gr.Dropdown(label="Splitter",
                                     choices=["chunk_size=1000, chunk_overlap=250",
                                              "chunk_size=500, chunk_overlap=50",
                                              "chunk_size=300, chunk_overlap=0"],
                                     value="chunk_size=1000, chunk_overlap=250"),
                         gr.Files(file_types=["pdf", "txt", "pptx", "csv", "html", "xlsx", "docx"],
                                  label="Load files to RAG"),
                         gr.Textbox(label="Question")],
                     outputs="text",
                     title="Document Query with Ollama",
                     description="Enter files and a question to query the documents.")

In [10]:
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://00e6262a488cd8b539.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 285, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1923, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1508, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

[INFO] split_documents
[DEBUG] [Document(metadata={'source': '/tmp/gradio/fb2317df183e37326b8378b7f120141680104372/Mystic.docx'}, page_content='An unofficial resurrection and reconstruction\n\nof the iconic psionic class option.\n\nSupernatural, not Magical\n\nAlways Free\n\nProof of Concept\n\nRespectful of Class Identity\n\nReducing the Bookkeeping\n\nCredits\n\nConsultants\n\nContents\n\nFonts\n\nDungeons & Dragons Legal Information\n\nHit Points\n\nProficiencies\n\nCreating a Mystic\n\nEquipment\n\nQuick Build\n\nConcentration\n\nPsionic Talents Known\n\nSpells as Psionic Talents\n\nPsionic Ability\n\nPsionic Ability Modifier\n\nPsionic save DC = 8 + your proficiency bonus + your Wisdom\n\nmodifier\n\nPsionic attack modifier = your proficiency bonus + your Wisdom\n\nmodifier\n\nOrder Talents\n\nDeathspeaker\n\nSpirit Consort\n\nExtrasensory Investigation\n\nSoul Sanctum\tOrder Talents\n\nForeteller\n\nImmortal Will\n\nSecond Sight\n\nBreadth of Knowledge\n\nPrescience\n\nPsionic Ab

[INFO] split_documents
[DEBUG] [Document(metadata={'source': '/tmp/gradio/b6271c750f89acd42251e2fd6ba8b7d3ca6ce25c/Ценообразование.pptx'}, page_content='ЭКОНОМИЧЕСКОЕ СОДЕРЖАНИЕ ЦЕНЫ\n\nПроцессы ценообразования оказывают влияние не только на хозяйственные процессы, но и на социальные процессы. Цены оказывают влияние на результаты хозяйственной деятельности предприятия.\n\nЦена – это денежное выражение стоимости товара, услуги, выполненной работы.\n\nЦенообразование – это процесс установления цены.\n\nСуществуют две основные системы ценообразования (система ценообразования включает систему цен и механизм ценообразования): \n\nзатратное ценообразование (цена зависит от себестоимости произведен-ной продукции);\n\nрыночное ценообразование (цена определяется соотношением спроса и предложения, издержками производства, полезностью, зависит от типа рынка (свободной конкуренции, монополистической конкуренции, др.; цена – это сумма денег, за которую продавец готов продать продукцию, а покупатель